<a href="https://colab.research.google.com/github/Van-Wu1/cycle/blob/main/scr/py/s1_tmdslope.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install geopandas shapely rasterio pyproj tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 86.4 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')
!ls '/content/drive/MyDrive/CASA0004_Cycling/data'

Mounted at /content/drive
BoroughShp  GreatLondonShp  s1	s2_Env	s3


In [3]:
import math
import numpy as np
import rasterio
from shapely.geometry import LineString, MultiLineString
from tqdm import tqdm

import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from shapely.strtree import STRtree
import numpy as np
import matplotlib.pyplot as plt
import os
import glob

In [4]:
# 输入文件夹和输出文件夹
input_dir = "/content/drive/MyDrive/CASA0004_Cycling/data/s1/Roads_OT/CQI_1"
output_dir = "/content/drive/MyDrive/CASA0004_Cycling/data/s1/Roads_OT/CQI_final"

# 确保输出文件夹存在
os.makedirs(output_dir, exist_ok=True)

In [5]:
# 可调参数
SLOPE_TIF = "/content/drive/MyDrive/CASA0004_Cycling/data/s1/Slope/slope.tif"
INTERVAL_M = 5.0             # 采样间距（米）
STAT_KIND  = "q3"             # "q3" | "mean" | "max"
SLOPE_UNIT = "degree"         # 栅格单位 or "percent"
CHUNK_SIZE = 20000            # 分块行数，视内存调整

In [6]:
def line_points_every(line: LineString, step: float):
    """沿 LineString 每 step 米取点（含终点）。"""
    if not line or line.length == 0:
        return []
    n = max(1, int(math.floor(line.length / step)))
    dists = [i * step for i in range(n)] + [line.length]
    return [line.interpolate(d).coords[0] for d in dists]

def geom_points(geom, step: float):
    if geom is None:
        return []
    if isinstance(geom, LineString):
        return line_points_every(geom, step)
    if isinstance(geom, MultiLineString):
        pts = []
        for part in geom.geoms:
            pts.extend(line_points_every(part, step))
        return pts
    return []

def calc_stat(vals, kind="q3"):
    if not vals:
        return 0.0
    arr = np.asarray(vals, dtype=float)
    if kind == "mean": return float(np.nanmean(arr))
    if kind == "max":  return float(np.nanmax(arr))
    return float(np.nanpercentile(arr, 75))  # q3

def slope_to_percent(v, unit="degree"):
    if unit == "percent": return float(v)
    return float(math.tan(math.radians(v)) * 100.0)

def slope_to_factor(slope_pct: float) -> float:
    if slope_pct <= 2:   return 1.0
    if slope_pct <= 4:   return 0.95
    if slope_pct <= 6:   return 0.85
    if slope_pct <= 8:   return 0.75
    if slope_pct <= 10:  return 0.65
    return 0.9

# LTS的映射字典
def lts_to_factor(lts: float) -> float:
    if lts == 1: return 1.0
    if lts == 2: return 0.9
    if lts == 3: return 0.75
    if lts == 4: return 0.6
    return 0.9 #没关系这里没有空

In [7]:
# 插入合并代码（全流程优化中）
# 遍历所有 geojson 文件
geojson_files = sorted(glob.glob(os.path.join(input_dir, "*.geojson")))

for file_path in geojson_files:
    print(f"正在处理：{file_path}")

    # 读取文件
    road_gdf = gpd.read_file(file_path)

    # TODO: 数据处理逻辑
    # # 打开坡度栅格
    ds = rasterio.open(SLOPE_TIF)
    raster_crs = ds.crs
    nodata = ds.nodata

    # 确保道路与栅格同一 CRS
    if road_gdf.crs is None:
        raise RuntimeError("road_gdf 没有 CRS，请先设定。")
    if raster_crs and road_gdf.crs.to_wkt() != raster_crs.to_wkt():
        road_gdf = road_gdf.to_crs(raster_crs)

    # 建字段
    if "proc_slope" not in road_gdf.columns:
        road_gdf["proc_slope"] = np.nan
    if "fac_3" not in road_gdf.columns:
        road_gdf["fac_3"] = np.nan
    if "fac_5" not in road_gdf.columns:
        road_gdf["fac_5"] = np.nan

    total = len(road_gdf)
    pbar = tqdm(total=total, desc="Sampling slope", unit="feat")

    start = 0
    while start < total:
        end = min(start + CHUNK_SIZE, total)
        sub = road_gdf.iloc[start:end].copy()

        proc_vals = np.zeros(len(sub))
        fac_vals  = np.ones(len(sub))

        # 为每条线生成采样点 → 栅格采样 → 统计
        for i, geom in enumerate(sub.geometry.values):
            pts = geom_points(geom, INTERVAL_M)
            if not pts:
                proc_vals[i] = 0.0
                fac_vals[i]  = 1.0
                continue

            # rasterio.sample 需要 [(x,y), ...]
            smps = list(ds.sample(pts))
            vals = []
            for s in smps:
                if s is None or len(s) == 0:
                    continue
                v = s[0]
                if v is None:
                    continue
                if (nodata is not None and v == nodata) or np.isnan(v):
                    continue
                vals.append(float(v))

            stat_v = calc_stat(vals, STAT_KIND)
            slope_pct = slope_to_percent(stat_v, SLOPE_UNIT)
            factor = slope_to_factor(slope_pct)

            proc_vals[i] = round(slope_pct, 2)
            fac_vals[i]  = round(factor, 2)

        # 写回
        road_gdf.loc[sub.index, "proc_slope"] = proc_vals
        road_gdf.loc[sub.index, "fac_3"] = fac_vals

        pbar.update(len(sub))
        start = end

    pbar.close()

    road_gdf["fac_5"] = road_gdf["stress_level"].apply(lts_to_factor)

    road_gdf["index"] = road_gdf["base_index"] * road_gdf["fac_1"] * road_gdf["fac_2"] * road_gdf["fac_3"] * road_gdf["fac_4"] * road_gdf["fac_5"]

    road_gdf["index"] = road_gdf["index"].clip(lower=0, upper=100).round().astype(int)

    road_gdf["index_10"] = road_gdf["index"] // 10

    # 构造输出路径（保留原文件名）
    file_name = os.path.basename(file_path)
    output_path = os.path.join(output_dir, file_name)

    # 保存
    road_gdf.to_file(output_path, driver="GeoJSON")
    print(f"已保存到：{output_path}")

print("全部文件处理完成！")

正在处理：/content/drive/MyDrive/CASA0004_Cycling/data/s1/Roads_OT/CQI_1/1_CQI1.geojson


Sampling slope: 100%|██████████| 26443/26443 [01:02<00:00, 423.55feat/s]


已保存到：/content/drive/MyDrive/CASA0004_Cycling/data/s1/Roads_OT/CQI_final/1_CQI1.geojson
正在处理：/content/drive/MyDrive/CASA0004_Cycling/data/s1/Roads_OT/CQI_1/2_CQI1.geojson


Sampling slope: 100%|██████████| 28951/28951 [00:48<00:00, 600.90feat/s]


已保存到：/content/drive/MyDrive/CASA0004_Cycling/data/s1/Roads_OT/CQI_final/2_CQI1.geojson
正在处理：/content/drive/MyDrive/CASA0004_Cycling/data/s1/Roads_OT/CQI_1/3_CQI1.geojson


...

已保存到：/content/drive/MyDrive/CASA0004_Cycling/data/s1/Roads_OT/CQI_final/7_CQI1.geojson
正在处理：/content/drive/MyDrive/CASA0004_Cycling/data/s1/Roads_OT/CQI_1/8_CQI1.geojson


Sampling slope: 100%|██████████| 44215/44215 [01:00<00:00, 733.22feat/s]


已保存到：/content/drive/MyDrive/CASA0004_Cycling/data/s1/Roads_OT/CQI_final/8_CQI1.geojson
正在处理：/content/drive/MyDrive/CASA0004_Cycling/data/s1/Roads_OT/CQI_1/9_CQI1.geojson


Sampling slope: 100%|██████████| 16178/16178 [00:31<00:00, 516.41feat/s]


已保存到：/content/drive/MyDrive/CASA0004_Cycling/data/s1/Roads_OT/CQI_final/9_CQI1.geojson
全部文件处理完成！


In [ ]:
road_gdf = gpd.read_file("/content/drive/MyDrive/CASA0004_Cycling/data/Roads/AAA.geojson")
print(road_gdf.columns)

In [ ]:
def line_points_every(line: LineString, step: float):
    """沿 LineString 每 step 米取点（含终点）。"""
    if not line or line.length == 0:
        return []
    n = max(1, int(math.floor(line.length / step)))
    dists = [i * step for i in range(n)] + [line.length]
    return [line.interpolate(d).coords[0] for d in dists]

def geom_points(geom, step: float):
    if geom is None:
        return []
    if isinstance(geom, LineString):
        return line_points_every(geom, step)
    if isinstance(geom, MultiLineString):
        pts = []
        for part in geom.geoms:
            pts.extend(line_points_every(part, step))
        return pts
    return []

def calc_stat(vals, kind="q3"):
    if not vals:
        return 0.0
    arr = np.asarray(vals, dtype=float)
    if kind == "mean": return float(np.nanmean(arr))
    if kind == "max":  return float(np.nanmax(arr))
    return float(np.nanpercentile(arr, 75))  # q3

def slope_to_percent(v, unit="degree"):
    if unit == "percent": return float(v)
    return float(math.tan(math.radians(v)) * 100.0)

def slope_to_factor(slope_pct: float) -> float:
    if slope_pct <= 2:   return 1.0
    if slope_pct <= 4:   return 0.95
    if slope_pct <= 6:   return 0.85
    if slope_pct <= 8:   return 0.75
    if slope_pct <= 10:  return 0.65
    return 0.9

In [ ]:
# 打开坡度栅格
ds = rasterio.open(SLOPE_TIF)
raster_crs = ds.crs
nodata = ds.nodata

# 确保道路与栅格同一 CRS
if road_gdf.crs is None:
    raise RuntimeError("road_gdf 没有 CRS，请先设定。")
if raster_crs and road_gdf.crs.to_wkt() != raster_crs.to_wkt():
    road_gdf = road_gdf.to_crs(raster_crs)

# 建字段
if "proc_slope" not in road_gdf.columns:
    road_gdf["proc_slope"] = np.nan
if "fac_3" not in road_gdf.columns:
    road_gdf["fac_3"] = np.nan


In [ ]:
total = len(road_gdf)
pbar = tqdm(total=total, desc="Sampling slope", unit="feat")

start = 0
while start < total:
    end = min(start + CHUNK_SIZE, total)
    sub = road_gdf.iloc[start:end].copy()

    proc_vals = np.zeros(len(sub))
    fac_vals  = np.ones(len(sub))

    # 为每条线生成采样点 → 栅格采样 → 统计
    for i, geom in enumerate(sub.geometry.values):
        pts = geom_points(geom, INTERVAL_M)
        if not pts:
            proc_vals[i] = 0.0
            fac_vals[i]  = 1.0
            continue

        # rasterio.sample 需要 [(x,y), ...]
        smps = list(ds.sample(pts))
        vals = []
        for s in smps:
            if s is None or len(s) == 0:
                continue
            v = s[0]
            if v is None:
                continue
            if (nodata is not None and v == nodata) or np.isnan(v):
                continue
            vals.append(float(v))

        stat_v = calc_stat(vals, STAT_KIND)
        slope_pct = slope_to_percent(stat_v, SLOPE_UNIT)
        factor = slope_to_factor(slope_pct)

        proc_vals[i] = round(slope_pct, 2)
        fac_vals[i]  = round(factor, 2)

    # 写回
    road_gdf.loc[sub.index, "proc_slope"] = proc_vals
    road_gdf.loc[sub.index, "fac_3"] = fac_vals

    pbar.update(len(sub))
    start = end

pbar.close()


In [ ]:
print(road_gdf.columns)

In [ ]:
print("CRS:", ds.crs)
print("Width (cols):", ds.width)
print("Height (rows):", ds.height)
print("Transform:", ds.transform)
print("Pixel size (res):", ds.res)


In [ ]:
# 必要的copy
copy = road_gdf

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 1. 基本统计信息
print(road_gdf['fac_3'].describe())

# 2. 直方图 + 核密度曲线
sns.histplot(road_gdf['fac_3'], kde=True, bins=20)
plt.title('Distribution of fac_3')
plt.xlabel('fac_3')
plt.ylabel('Frequency')
plt.show()

# 3. 箱线图（看中位数和异常值）
sns.boxplot(x=road_gdf['fac_3'])
plt.title('Boxplot of fac_3')
plt.show()


In [ ]:
# 1. 基本统计信息
print(road_gdf['proc_slope'].describe())

# 2. 直方图 + 核密度曲线
sns.histplot(road_gdf['proc_slope'], kde=True, bins=20)
plt.title('Distribution of proc_slope')
plt.xlabel('proc_slope')
plt.ylabel('Frequency')
plt.show()

# 3. 箱线图（看中位数和异常值）
sns.boxplot(x=road_gdf['proc_slope'])
plt.title('Boxplot of proc_slope')
plt.show()


In [ ]:
print("CRS:", ds.crs)
print("Width (cols):", ds.width)
print("Height (rows):", ds.height)
print("Transform:", ds.transform)
print("Pixel size (res):", ds.res)

# index计算&覆写

In [ ]:
# road_gdf["index"] = road_gdf["base_index"] * road_gdf["fac_1"] * road_gdf["fac_2"] * road_gdf["fac_3"] * road_gdf["fac_4"]

# road_gdf["index"] = road_gdf["index"].clip(lower=0, upper=100).round().astype(int)

# road_gdf["index_10"] = road_gdf["index"] // 10

# 然后根据我本子上的三条id进行核对，没问题就转到下一步去加入LTS的参数(没问题了)

In [ ]:
#output_path = "/content/drive/MyDrive/CASA0004_Cycling/data/Roads/AAA_indexupdate.geojson"
#road_gdf.to_file(output_path, driver="GeoJSON")
#print(f"已保存到: {output_path}")

## 小加一波LTS

In [ ]:
# 先检查一下lts是否有空值

# 统计空值数量
null_count = road_gdf["stress_level"].isna().sum()
print(f"空值数量: {null_count}")

# 具体空值的行
null_rows = road_gdf[road_gdf["stress_level"].isna()]
print(null_rows)

# 很好没有空值

In [ ]:
# LTS的映射字典
def lts_to_factor(lts: float) -> float:
    if lts == 1: return 1.0
    if lts == 2: return 0.9
    if lts == 3: return 0.75
    if lts == 4: return 0.6
    return 0.9 #没关系这里没有空

In [ ]:
# 建fac_5字段用来装LTS的量化
if "fac_5" not in road_gdf.columns:
    road_gdf["fac_5"] = np.nan

In [ ]:
# ok我们来试一下
road_gdf["fac_5"] = road_gdf["stress_level"].apply(lts_to_factor)

In [ ]:
print(road_gdf.columns)

In [ ]:
road_gdf["index"] = road_gdf["base_index"] * road_gdf["fac_1"] * road_gdf["fac_2"] * road_gdf["fac_3"] * road_gdf["fac_4"] * road_gdf["fac_5"]

road_gdf["index"] = road_gdf["index"].clip(lower=0, upper=100).round().astype(int)

road_gdf["index_10"] = road_gdf["index"] // 10

In [ ]:
output_path_fac5 = "/content/drive/MyDrive/CASA0004_Cycling/data/Roads/AAA_allfac.geojson"
road_gdf.to_file(output_path_fac5, driver="GeoJSON")
print(f"已保存到: {output_path_fac5}")

### 看一眼情况

In [ ]:
print(road_gdf["index"].describe())

In [ ]:
print(road_gdf["index"].value_counts().sort_index())

In [ ]:
print(road_gdf["index"].value_counts(bins=10).sort_index())

In [ ]:
road_gdf["index"].hist(bins=20)
plt.xlabel("Index")
plt.ylabel("Frequency")
plt.title("Index Distribution")
plt.show()

In [ ]:
# 我们来比对一下
road_gdf_origin = gpd.read_file("/content/drive/MyDrive/CASA0004_Cycling/data/Roads/AAA.geojson")
print(road_gdf_origin.columns)

road_gdf_origin["index"].hist(bins=20)
plt.xlabel("Index")
plt.ylabel("Frequency")
plt.title("Index Distribution")
plt.show()

In [ ]:
road_gdf["id"].value_counts().head(10)          # 新数据里重复最多的 id

In [ ]:
road_gdf_origin["id"].value_counts().head(10)   # 旧数据里重复最多的 id

In [ ]:
import geopandas as gpd
import pandas as pd

# 1) 只取需要的列并改名，方便区分
g_new = road_gdf[["id", "index", "geometry"]].rename(columns={"index":"index_new"})
g_old = road_gdf_origin[["id", "index"]].rename(columns={"index":"index_old"})

# （可选）如果CRS不一致，统一一下
if g_new.crs != road_gdf_origin.crs:
    g_new = g_new.to_crs(road_gdf_origin.crs)

# 2) 按 id 合并，并计算差值（新-旧：>0代表分数上升，<0代表下降）
g_diff = g_new.merge(g_old, on="id", how="inner")
g_diff["index_diff"] = g_diff["index_new"] - g_diff["index_old"]

# 3) 快速看统计与变化最大的要素
print(g_diff["index_diff"].describe())
print("Top↑ 提升最大的10条：")
print(g_diff.nlargest(10, "index_diff")[["id", "index_old", "index_new", "index_diff"]])
print("Top↓ 降幅最大的10条：")
print(g_diff.nsmallest(10, "index_diff")[["id", "index_old", "index_new", "index_diff"]])

# 4) 静态差值图（红降蓝升），需要 mapclassify 才能用 scheme="quantiles"
ax = g_diff.plot(column="index_diff",
                 cmap="RdBu_r",  # 蓝=上升，红=下降（_r 代表反转）
                 legend=True,
                 linewidth=0.1)
ax.set_title("Index Difference (new - old)")
ax.set_axis_off()

# 5) 导出方便在GIS中做热力对比
out_path = "/content/drive/MyDrive/CASA0004_Cycling/data/Roads/AAA_index_diff.geojson"
g_diff.to_file(out_path, driver="GeoJSON")
print(f"差值图层已导出：{out_path}")


In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import folium
from shapely.geometry import LineString

# ========= 0) 生成差值图层（若你已有 g_diff 可跳过这块） =========
# 假设已有 road_gdf（新）与 road_gdf_origin（旧），都含 id 与 geometry
g_new = road_gdf[["id", "index", "geometry"]].rename(columns={"index":"index_new"})
g_old = road_gdf_origin[["id", "index"]].rename(columns={"index":"index_old"})

# 统一CRS
if g_new.crs != road_gdf_origin.crs:
    g_new = g_new.to_crs(road_gdf_origin.crs)

g_diff = g_new.merge(g_old, on="id", how="inner")
g_diff["index_diff"] = g_diff["index_new"] - g_diff["index_old"]

print("== index_diff 概览 ==")
print(g_diff["index_diff"].describe())


In [ ]:
# ========= 1) 区间分布（value_counts with bins） =========
# 自动根据数据范围构造对称分箱（步长=5分，可改）
dmin = np.floor(g_diff["index_diff"].min())
dmax = np.ceil(g_diff["index_diff"].max())
bound = int(max(abs(dmin), abs(dmax)))
step = 5
bins = np.arange(-bound - (bound % step), bound + step, step)  # 对称到零两侧

vc = g_diff["index_diff"].value_counts(bins=bins, sort=False)
dist_df = pd.DataFrame({
    "bin": vc.index.astype(str),
    "count": vc.values
})
dist_df["percent"] = (dist_df["count"] / len(g_diff) * 100).round(2)
print("== index_diff 区间分布（步长=5）==")
print(dist_df)

In [ ]:
# ========= 2) 按 Borough 聚合平均变化 =========
# 👉 将路径替换为你的 Borough 边界文件（GeoJSON/GeoPackage/Shapefile 都可）
BORO_PATH = "/content/drive/MyDrive/CASA0004_Cycling/data/Admin/London_Borough_Boundaries.geojson"
boroughs = gpd.read_file(BORO_PATH)

# 统一CRS后做空间连接
if boroughs.crs != g_diff.crs:
    boroughs = boroughs.to_crs(g_diff.crs)

# 可能的名称字段
name_cols = [c for c in boroughs.columns if c.lower() in ("name", "borough", "borough_name", "la_name")]
if not name_cols:
    raise ValueError("在 Borough 图层中没有找到名称字段（name/borough/borough_name/la_name），请检查并修改代码。")
BORO_NAME_COL = name_cols[0]

# 空间连接，将每条线归到所在 Borough（交集即可）
g_join = gpd.sjoin(g_diff, boroughs[[BORO_NAME_COL, "geometry"]], how="left", predicate="intersects")
boro_agg = (
    g_join
    .groupby(BORO_NAME_COL)
    .agg(
        mean_index_diff=("index_diff", "mean"),
        median_index_diff=("index_diff", "median"),
        n_links=("id", "count")
    )
    .sort_values("mean_index_diff", ascending=False)
    .round(2)
)
print("== Borough 平均变化（由高到低）==")
print(boro_agg.head(10))
print("== Borough 平均变化（下降最明显）==")
print(boro_agg.tail(10))

# 导出 Borough 聚合表与带属性的区划
boroughs_out = boroughs.merge(boro_agg, on=BORO_NAME_COL, how="left")
boroughs_csv = "/content/drive/MyDrive/CASA0004_Cycling/data/Roads/borough_index_diff_summary.csv"
boroughs_gj  = "/content/drive/MyDrive/CASA0004_Cycling/data/Roads/borough_index_diff.geojson"
boro_agg.to_csv(boroughs_csv, encoding="utf-8")
boroughs_out.to_file(boroughs_gj, driver="GeoJSON")
print(f"Borough 聚合已导出:\n- {boroughs_csv}\n- {boroughs_gj}")

# ========= 3) 只渲染显著变化（|diff| >= 5）到 Folium =========
sig = g_diff.loc[g_diff["index_diff"].abs() >= 5].copy()
print(f"显著变化要素数: {len(sig)}")

# Folium 需要 WGS84
sig_4326 = sig.to_crs(epsg=4326)

# 简单的颜色映射：上升=蓝，下降=红
def color_by_diff(d):
    return "#2b8cbe" if d >= 0 else "#de2d26"

# 可选：限制最大渲染数量，避免浏览器过重（比如只取|diff|最大的2万条）
MAX_FEATS = 20000
if len(sig_4326) > MAX_FEATS:
    sig_4326 = sig_4326.reindex(
        sig_4326["index_diff"].abs().sort_values(ascending=False).index[:MAX_FEATS]
    )

# 地图中心
if not sig_4326.empty:
    center = [sig_4326.geometry.iloc[0].centroid.y, sig_4326.geometry.iloc[0].centroid.x]
else:
    center = [51.5074, -0.1278]  # 伦敦

m = folium.Map(location=center, zoom_start=11, tiles="cartodbpositron")

# 上升与下降分两层
up = sig_4326[sig_4326["index_diff"] >= 0]
down = sig_4326[sig_4326["index_diff"] < 0]

def add_geojson_lines(gdf, name, color, map_obj, weight=3, opacity=0.7):
    folium.GeoJson(
        gdf.to_json(),
        name=name,
        style_function=lambda f: {"color": color, "weight": weight, "opacity": opacity},
        tooltip=folium.features.GeoJsonTooltip(
            fields=["id", "index_old", "index_new", "index_diff"],
            aliases=["id", "old", "new", "diff"],
            sticky=True
        )
    ).add_to(map_obj)

if not up.empty:
    add_geojson_lines(up, "Increase (>= +5)", "#2b8cbe", m)
if not down.empty:
    add_geojson_lines(down, "Decrease (<= -5)", "#de2d26", m)

folium.LayerControl().add_to(m)

html_out = "/content/drive/MyDrive/CASA0004_Cycling/data/Roads/AAA_index_diff_hotspots.html"
m.save(html_out)
print(f"显著变化热点地图已导出: {html_out}")

# ========= 4) 额外：导出显著变化矢量，便于GIS叠加 =========
sig_out = "/content/drive/MyDrive/CASA0004_Cycling/data/Roads/AAA_index_diff_significant.geojson"
sig.to_file(sig_out, driver="GeoJSON")
print(f"显著变化矢量已导出: {sig_out}")
